In [1]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install undetected-chromedriver


  Preparing metadata (setup.py) ... done
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47099 sha256=a27a62f2e9b6cfe27da7223c9d0c2ff91fa888cb4541e54134956d1aece7e003
  Stored in directory: /Users/rebeccahorst/Library/Caches/pip/wheels/c4/f1/aa/9de6cf276210554d91e9c0526864563e850a428c5e76da4914
Successfully built undetected-chromedriver
Note: you may need to restart the kernel to use updated packages.


In [7]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup

driver = uc.Chrome(headless=True)
driver.get("https://doge.gov/savings")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify()[:1000])


<html dir="ltr" lang="en-US">
 <head>
  <title>
   Just a moment...
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="noindex,nofollow" name="robots"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <style>
   *{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url(data:image/s

In [9]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd
import time

# Start undetected Chrome
driver = uc.Chrome(headless=True)

# Load the DOGE page
driver.get("https://doge.gov/savings")
time.sleep(8)  # wait for Cloudflare + JavaScript to finish loading

# Grab the full page source
html = driver.page_source
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Try finding all tables
tables = soup.find_all('table')

if not tables:
    print("⚠️ No <table> elements found. Looking for div-based grids or lists...")

    # Print first few "summary boxes" or list elements
    all_divs = soup.find_all('div')
    for div in all_divs:
        if any(x in div.text.lower() for x in ["billion", "million", "contract", "grant", "lease"]):
            print("—", div.text.strip()[:200])
else:
    for i, table in enumerate(tables):
        headers = [th.get_text(strip=True) for th in table.find_all('th')]
        rows = []
        for tr in table.find_all('tr')[1:]:
            cells = [td.get_text(strip=True) for td in tr.find_all('td')]
            if cells:
                rows.append(cells)
        df = pd.DataFrame(rows, columns=headers if headers else None)
        df.to_csv(f'doge_table_{i+1}.csv', index=False)
        print(f"✅ Saved doge_table_{i+1}.csv with {len(df)} rows")

⚠️ No <table> elements found. Looking for div-based grids or lists...


In [10]:
driver = uc.Chrome(headless=False)

In [13]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Start Chrome
driver = uc.Chrome(headless=False)
driver.get("https://doge.gov/savings")
time.sleep(10)  # Wait for initial load (Cloudflare, JS, etc.)

# Storage for rows
all_rows = []
headers = []
visited_pages = set()

# Helper to extract table on current page
def extract_first_table():
    global headers
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    if not table:
        print("⚠️ No table found.")
        return []

    if not headers:
        headers = [th.get_text(strip=True) for th in table.find_all('th')]

    rows = []
    for tr in table.find_all('tr')[1:]:
        cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        if cells:
            rows.append(cells)
    return rows

# Extract first page
print("📄 Extracting page 1")
all_rows += extract_first_table()
visited_pages.add("1")

# Start pagination loop
while True:
    time.sleep(1)
    pagination_buttons = driver.find_elements(By.XPATH, '//button[normalize-space(text()) and not(@disabled)]')
    page_texts = [btn.text.strip() for btn in pagination_buttons if btn.text.strip().isdigit()]
    next_pages = [p for p in page_texts if p not in visited_pages]

    if not next_pages:
        print("✅ No more pages to visit.")
        break

    next_page = next_pages[0]
    print(f"➡️ Navigating to page {next_page}")
    
    try:
        next_button = driver.find_element(By.XPATH, f'//button[normalize-space(text())="{next_page}"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)  # Wait for page update
        rows = extract_first_table()
        if rows:
            all_rows += rows
            visited_pages.add(next_page)
        else:
            print(f"⚠️ No rows found on page {next_page}")
            break
    except Exception as e:
        print(f"❌ Error clicking page {next_page}: {e}")
        break

driver.quit()

# Save to CSV
df = pd.DataFrame(all_rows, columns=headers if headers else None)
df.to_csv("doge_first_table_full.csv", index=False)
print(f"✅ Saved {len(df)} rows to doge_first_table_full.csv")


📄 Extracting page 1
➡️ Navigating to page 2
➡️ Navigating to page 3
➡️ Navigating to page 4
➡️ Navigating to page 5
➡️ Navigating to page 6
➡️ Navigating to page 7
➡️ Navigating to page 8
➡️ Navigating to page 9
➡️ Navigating to page 10
➡️ Navigating to page 11
➡️ Navigating to page 12
➡️ Navigating to page 13
➡️ Navigating to page 14
➡️ Navigating to page 15
➡️ Navigating to page 16
➡️ Navigating to page 17
➡️ Navigating to page 18
➡️ Navigating to page 19
➡️ Navigating to page 20
➡️ Navigating to page 21
➡️ Navigating to page 22
➡️ Navigating to page 23
➡️ Navigating to page 24
➡️ Navigating to page 25
➡️ Navigating to page 26
➡️ Navigating to page 27
➡️ Navigating to page 28
➡️ Navigating to page 29
➡️ Navigating to page 30
➡️ Navigating to page 31
➡️ Navigating to page 32
➡️ Navigating to page 33
➡️ Navigating to page 34
➡️ Navigating to page 35
➡️ Navigating to page 36
➡️ Navigating to page 37
➡️ Navigating to page 38
➡️ Navigating to page 39
➡️ Navigating to page 40
➡️ Navigating